<a href="https://colab.research.google.com/github/ngogadieudonne88/25011133_NGOGA-Dieudonne/blob/main/REGNUMBER_25011133_NgogaDieudonne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import time
import logging
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Configuration
CHECK_INTERVAL = 300  # 5 minutes in seconds
URLS_TO_MONITOR = [
    "https://example.com",
    "https://another-website.com",
    "https://uok.ac.rw"
]

# Email configuration
SMTP_SERVER = "smtp.gmail.com"  # Replace with your SMTP server
SMTP_PORT = 587  # Replace with your SMTP port
SENDER_EMAIL = "your-email@gmail.com"  # Replace with your email address
SENDER_PASSWORD = "your-email-password"  # Replace with your email password
RECIPIENT_EMAIL = "recipient-email@example.com"  # Replace with recipient email address

# Slack configuration
SLACK_WEBHOOK_URL = "https://hooks.slack.com/services/your/slack/webhook"  # Replace with your Slack webhook URL

# Logging configuration
logging.basicConfig(filename="website_monitor.log", level=logging.INFO,
                    format="%(asctime)s - %(levelname)s - %(message)s")

def send_email(subject, body):
    """Send an email notification."""
    try:
        msg = MIMEMultipart()
        msg["From"] = SENDER_EMAIL
        msg["To"] = RECIPIENT_EMAIL
        msg["Subject"] = subject
        msg.attach(MIMEText(body, "plain"))

        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
            server.starttls()
            server.login(SENDER_EMAIL, SENDER_PASSWORD)
            server.sendmail(SENDER_EMAIL, RECIPIENT_EMAIL, msg.as_string())
        logging.info("Email notification sent successfully.")
    except Exception as e:
        logging.error(f"Error sending email: {e}")

def send_slack_alert(message):
    """Send a Slack notification."""
    try:
        payload = {"text": message}
        response = requests.post(SLACK_WEBHOOK_URL, json=payload)
        if response.status_code != 200:
            logging.error(f"Slack alert failed: {response.status_code} - {response.text}")
        else:
            logging.info("Slack notification sent successfully.")
    except Exception as e:
        logging.error(f"Error sending Slack alert: {e}")

def check_website(url):
    """Check if a website is up and running."""
    try:
        response = requests.get(url, timeout=5)
        if response.status_code == 200:
            return f"Website {url} is up (Status: {response.status_code})"
        else:
            return f"Website {url} returned an unexpected status code: {response.status_code}"
    except requests.exceptions.RequestException as e:
        return f"Website {url} is unreachable. Error: {e}"

def monitor_websites():
    """Monitor multiple websites and send alerts if any are down."""
    while True:
        for url in URLS_TO_MONITOR:
            result = check_website(url)
            logging.info(result)
            print(result)  # Print to console for real-time monitoring

            # Send alerts if the website is down
            if "unreachable" in result.lower() or "unexpected" in result.lower():
                subject = f"Website Down Alert: {url}"
                body = f"The website {url} is down or unreachable.\n\nDetails: {result}"
                send_email(subject, body)
                send_slack_alert(f"🚨 {subject}\n{body}")

        # Wait for the next check
        time.sleep(CHECK_INTERVAL)

if __name__ == "__main__":
    print("Starting website monitoring...")
    monitor_websites()

Starting website monitoring...
Website https://example.com is up (Status: 200)
Website https://another-website.com is unreachable. Error: HTTPSConnectionPool(host='another-website.com', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7a0bd52ce690>: Failed to resolve 'another-website.com' ([Errno -2] Name or service not known)"))


ERROR:root:Error sending email: (535, b'5.7.8 Username and Password not accepted. For more information, go to\n5.7.8  https://support.google.com/mail/?p=BadCredentials d9443c01a7336-2230a0a619fsm17387105ad.195 - gsmtp')
ERROR:root:Slack alert failed: 404 - no_team


Website https://uok.ac.rw is up (Status: 200)
Website https://example.com is up (Status: 200)
Website https://another-website.com is unreachable. Error: HTTPSConnectionPool(host='another-website.com', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7a0bd52bbad0>: Failed to resolve 'another-website.com' ([Errno -2] Name or service not known)"))


ERROR:root:Error sending email: (535, b'5.7.8 Username and Password not accepted. For more information, go to\n5.7.8  https://support.google.com/mail/?p=BadCredentials 41be03b00d2f7-aeda79a30f4sm1672346a12.22 - gsmtp')
ERROR:root:Slack alert failed: 404 - no_team


Website https://uok.ac.rw is up (Status: 200)
